## Module 4.2: Residual Blocks/Connections

We will see how to create residual blocks/connections layers for use in advanced convolutional neural networks.

We will:
- Implement basic and convoluted residual blocks. In the first case we will look at connections skipping single or multiple layers.
- Create basic models and visually examine the residual architecture.

Since we are looking at *layers* rather than networks, we will not solve a problem in this module. However it is a good exercise for you to try on your own to make use of these ideas to improve the CNN performance from module 2.2.

We import desired libraries.

In [1]:
from keras.models import Model
from keras.layers import Input,Conv2D,Flatten,Dense,Add,BatchNormalization,Activation
from keras.utils import plot_model

import numpy as np

2024-02-28 12:19:13.892473: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


We will implement a function to create a block of simple residual layers.

In [2]:
# Since all layers will have the same number of filters to ensure element wise
# addition we could do this without a vector of filter numbers. But it will help
# to use such vectors once we generalize.
def simple_residual_block (layer_in,filters):
  # Define first convolution layer
  prev_layer = Conv2D(filters[0], (3,3), padding='same', activation='relu')(layer_in)
    
  for f in range(1,len(filters)):
    # Define later convolution layers
    conv = Conv2D(filters[f], (3,3), padding='same', activation='relu')(prev_layer)
    # Define skip connection (& update prev_layer)
    prev_layer = Add()([prev_layer, conv]) 
  
  return prev_layer

Here we design a simple network with such a residual block, and plot the network architecture.

In [4]:
# Define input
inputs = Input(shape=(128, 128, 3))
# Define residual block
res_block=simple_residual_block(inputs,[128,128,128])
# Flatten for output
flat = Flatten()(res_block)
dense1 = Dense(256, activation='relu')(flat)
outputs = Dense(10, activation='softmax')(dense1)
# Create model
model1 = Model(inputs=inputs, outputs=outputs)
# Summarize model
model1.summary()
# Plot model graph
plot_model(model1, show_shapes=True, to_file='residual_module.png')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_3 (Conv2D)              (None, 128, 128, 12  3584        ['input_2[0][0]']                
                                8)                                                                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 128, 128, 12  147584      ['conv2d_3[0][0]']               
                                8)                                                          

Where we have multiple layers in a basic block - such as a convolution + batch normalization + activation set - we would skip the block. The same is true of convolution + pooling or convolution + dropout layer sets (though care has to be taken to ensure that the resulting outputs match in size, and these are not so commonly used in residual networks).

Let's redo our wrapper function to implement a block of residual layers with convolution + batch normalization + activation.

In [5]:
# Since all layers will have the same number of filters to ensure element wise
# addition we could do this without a vector of filter numbers. But it will help
# to use such vectors once we generalize.
def batchnormed_residual_block (layer_in,filters):
  # Define first convolution layer
  prev_layer = Conv2D(filters[0], (3,3), padding='same', activation='relu')(layer_in)
    
  for f in range(1,len(filters)):
    # Define later convolution layers
    conv = Conv2D(filters[f], (3,3), padding='same')(prev_layer)
    batch = BatchNormalization()(conv)
    act = Activation("relu")(batch)
    # Define skip connection (& update prev_layer)
    prev_layer = Add()([prev_layer, act]) 
  
  return prev_layer

And let's design another simple network with two of these new residual blocks, and plot the network architecture.



In [6]:
# Define input
inputs = Input(shape=(128, 128, 3))
# Define first residual block
res_block1 = batchnormed_residual_block(inputs,[128,128,128])
# Define second residual block
res_block2 = batchnormed_residual_block(res_block1,[64,64,64])
# Flatten for output
flat = Flatten()(res_block2)
dense1 = Dense(256, activation='relu')(flat)
outputs = Dense(10, activation='softmax')(dense1)
# Create model
model1 = Model(inputs=inputs, outputs=outputs)
# Summarize model
model1.summary()
# Plot model graph
plot_model(model1, show_shapes=True, to_file='residual_module.png')

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 128, 128, 12  3584        ['input_3[0][0]']                
                                8)                                                                
                                                                                                  
 conv2d_7 (Conv2D)              (None, 128, 128, 12  147584      ['conv2d_6[0][0]']               
                                8)                                                          

We may also want to skip sets of convolution (or convolution + other sets) layers. So let's design a wrapper for such a purpose.

In [7]:
# Since all layers will have the same number of filters to ensure element wise
# addition we could do this without a vector of filter numbers. But it will help
# to use such vectors once we generalize.
def convset_residual_block (layer_in,filters,n):
  # Make internal function for defining set of convolution layers
  def make_conv_set(layer,f_):
    for c in range(n):
      layer = Conv2D(filters[f_], (3,3), padding='same', activation='relu')(layer)
    return layer

  # Define first set of convolution layers
  prev_layer=make_conv_set(layer_in,0)
    
  for f in range(1,len(filters)):
    # Define later set of convolution layers
    conv_set = make_conv_set(prev_layer,f)
    # Define skip connection (& update prev_layer)
    prev_layer = Add()([prev_layer, conv_set]) 
  
  return prev_layer

And let's design another simple network with two of these new residual blocks, and plot the network architecture.

In [8]:
inputs = Input(shape=(128, 128, 3))
# Define first *set* of convolution layers
conv1 = convset_residual_block(inputs,[128,128,128],2)
# Define second *set* of convolution layers
conv2 = convset_residual_block(conv1,[64,64,64],2)
# Flatten for output
flat = Flatten()(conv2)
dense1 = Dense(256, activation='relu')(flat)
outputs = Dense(10, activation='softmax')(dense1)
# Create model
model1 = Model(inputs=inputs, outputs=outputs)
# Summarize model
model1.summary()
# Plot model graph
plot_model(model1, show_shapes=True, to_file='residual_module.png')

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_12 (Conv2D)             (None, 128, 128, 12  3584        ['input_4[0][0]']                
                                8)                                                                
                                                                                                  
 conv2d_13 (Conv2D)             (None, 128, 128, 12  147584      ['conv2d_12[0][0]']              
                                8)                                                          

In our last example we create a wrapper function to define a residual block of convolution blocks where the shape of the output has changed. We add a convolution layer to the skip connection designed to suitably shape it for the desired addition. 

In [9]:
# Filters will be a 2d numpy array, giving filters for each convolution layer
# for each block (rows=block,columns=filter)
def dimchanged_convset_residual_block (layer_in,filters):
  # Make internal function for defining set of convolution layers
  def make_conv_set(layer,f_):
    for i in range(filters.shape[1]):
      layer = Conv2D(filters[f_,i], (3,3), activation='relu')(layer)
    return layer

  # Define first set of convolution layers
  prev_layer=make_conv_set(layer_in,0)
  
  # Work out dimensionality of skip convolution filter
  # (We know all filters in normal convolution layers are 3x3)
  skip_filter_dim=(2*filters.shape[1]+1,2*filters.shape[1]+1)
    
  for f in range(1,len(filters)):
    # Define later set of convolution layers
    conv_set = make_conv_set(prev_layer,f)
    # Define skip connection (& update prev_layer)
    # We need a convolution layer designed to match shapes    
    skip_conv = Conv2D(filters[f,-1], skip_filter_dim, activation='relu')(prev_layer)
    prev_layer = Add()([conv_set, skip_conv]) 
  
  return prev_layer

And we create another simple network with one of these new re-sizing residual blocks, and plot the network architecture.

In [10]:
inputs = Input(shape=(128, 128, 3))
# Define residual block of blocks of convolution layers that alter dimensionality.
filters=np.array([[64,128,256],[256,256,128]])
res_block=dimchanged_convset_residual_block(inputs,filters)
# Flatten for output
flat = Flatten()(res_block)
dense1 = Dense(256, activation='relu')(flat)
outputs = Dense(10, activation='softmax')(dense1)
# Create model
model1 = Model(inputs=inputs, outputs=outputs)
# Summarize model
model1.summary()
# Plot model graph
plot_model(model1, show_shapes=True, to_file='residual_module.png')

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_24 (Conv2D)             (None, 126, 126, 64  1792        ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 124, 124, 12  73856       ['conv2d_24[0][0]']              
                                8)                                                          